Автор: Лейман М.А.   
Дата создания: 22.03.2025  

# Обработка датасета

### Классы датасета  
'building': 0,      		здание  
'cable-tower': 1, 		кабельная вышка  
'cultivation-mesh-cage': 2,  	сетка для выращивания  
'landslide': 3, 		оползень  
'pool': 4, 			бассейн  
'prefabricated-house': 5, 	сборный-дом"  
'quarry': 6,			карьер  
'ship': 7, 			корабль  
'vehicle': 8,			транспортное средство  
'well': 9			колодец  



In [1]:
# Названия классов
class_names = ["building", "кабельная-вышка", "cultivation-mesh-cage", "landslide",
               "pool", "prefabricated-house", "quarry", "ship", "vehicle", "well"]


class_names2 = ["здание", "кабельная вышка", "сетка для выращивания", "оползень",
               "бассейн", "сборный-дом", "карьер", "корабль", "транспортное средство", "колодец"]

## Преобразование датасета в формат YOLO

In [ ]:
import os
import xml.etree.ElementTree as ET

# Пути к папкам
xml_dir = "dataset/target"  # Где лежат XML
output_txt_dir = "dataset/target_yolo"  # YOLO разметка
os.makedirs(output_txt_dir, exist_ok=True)


def convert_voc_to_yolo(xml_file, output_dir):
    """ 
        Конвертирует датасет в YOLO формат
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Получаем размеры изображения
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    yolo_labels = []
    for obj in root.findall("object"):
        class_name = obj.find("name").text
        if class_name not in class_names:
            continue
        class_id = class_names.index(class_name)

        bbox = obj.find("bndbox")
        xmin, ymin, xmax, ymax = map(int, [bbox.find("xmin").text, bbox.find("ymin").text,
                                           bbox.find("xmax").text, bbox.find("ymax").text])

        # Нормализация координат
        x_center = (xmin + xmax) / (2 * width)
        y_center = (ymin + ymax) / (2 * height)
        bbox_width = (xmax - xmin) / width
        bbox_height = (ymax - ymin) / height

        yolo_labels.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

    # Записываем в файл
    filename = os.path.basename(xml_file).replace(".xml", ".txt")
    with open(os.path.join(output_dir, filename), "w") as f:
        f.write("\n".join(yolo_labels))



# Обработаем датасет

# Обрабатываем все XML
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith(".xml"):
        convert_voc_to_yolo(os.path.join(xml_dir, xml_file), output_txt_dir)

print("Конвертация завершена!")

### Разделение датасета  
train  
val  
test  


In [ ]:
import os
import shutil
import random

def splitting_dataset(image_dir="datasets_full/images/train/", label_dir="datasets_full/labels/train" ):
    """ 
        Разделяет датасет
    """

    # Пути для разделённых данных
    output_dirs = {
        "train": ("datasets/images/train", "datasets/labels/train"),
        "val": ("datasets/images/val", "datasets/labels/val"),
        "test": ("datasets/images/test", "datasets/labels/test"),
    }

    # Создаем папки, если их нет
    for dirs in output_dirs.values():
        os.makedirs(dirs[0], exist_ok=True)  # images
        os.makedirs(dirs[1], exist_ok=True)  # labels

    # Получаем список файлов
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))])

    # Перемешиваем данные
    random.shuffle(image_files)

    # Разделение: 70% train, 20% val, 10% test
    train_split = int(0.7 * len(image_files))
    val_split = int(0.9 * len(image_files))

    splits = {
        "train": image_files[:train_split],
        "val": image_files[train_split:val_split],
        "test": image_files[val_split:],
    }

    # Копируем файлы
    for split, files in splits.items():
        img_out, lbl_out = output_dirs[split]

        for file in files:
            shutil.copy(os.path.join(image_dir, file), os.path.join(img_out, file))

            # Копируем аннотации (YOLO .txt или XML)
            label_file = os.path.splitext(file)[0] + ".txt"
            if os.path.exists(os.path.join(label_dir, label_file)):
                shutil.copy(os.path.join(label_dir, label_file), os.path.join(lbl_out, label_file))

    print("Датасет успешно разделён!")


In [5]:
splitting_dataset(image_dir="datasets_full/images/train/", label_dir="datasets_full/labels/train" )

Датасет успешно разделён!


### Аугментация

In [17]:
import os
import cv2
import numpy as np
from glob import glob

# Пути к датасету
input_images_dir = "datasets/images/train"
input_labels_dir = "datasets/labels/train"
output_images_dir = "datasets_augmented/images/train"
output_labels_dir = "datasets_augmented/labels/train"

# Создаем выходные папки
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)


def clip_bbox(bboxes):
    """Ограничивает координаты bbox в диапазоне [0,1]"""
    clipped_bboxes = []
    for box in bboxes:
        class_id, x, y, w, h = box
        x = max(0, min(1, x))
        y = max(0, min(1, y))
        w = max(0, min(1, w))
        h = max(0, min(1, h))
        clipped_bboxes.append([class_id, x, y, w, h])
    return clipped_bboxes


def adjust_brightness(image, factor=1.2):
    """Изменение яркости изображения"""
    return np.clip(image * factor, 0, 255).astype(np.uint8)


def add_noise(image, mean=0, std=25):
    """Добавление случайного гауссовского шума"""
    noise = np.random.normal(mean, std, image.shape).astype(np.int16)  # int16, чтобы избежать переполнения
    noisy_image = np.clip(image.astype(np.int16) + noise, 0, 255)  # Обрезаем в диапазон [0, 255]
    return noisy_image.astype(np.uint8)


def flip_image(image, bboxes, flip_code):
    """Отражение изображения и корректировка аннотаций"""
    h, w = image.shape[:2]
    flipped_img = cv2.flip(image, flip_code)
    
    new_bboxes = []
    for box in bboxes:
        class_id, x, y, bw, bh = box
        if flip_code == 1:  # Горизонтальное отражение
            x = 1 - x
        elif flip_code == 0:  # Вертикальное отражение
            y = 1 - y
        new_bboxes.append([class_id, x, y, bw, bh])
    
    return flipped_img, new_bboxes


def rotate_image(image, bboxes, angle=15):
    """Поворот изображения и корректировка аннотаций"""
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_img = cv2.warpAffine(image, matrix, (w, h))
    
    # Аннотации НЕ меняем (нужно пересчитывать координаты)
    return rotated_img, bboxes


def scale_image2(image, bboxes, scale_factor):
    """Масштабирование изображения"""
    h, w = image.shape[:2]
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    resized_img = cv2.resize(image, (new_w, new_h))

    # Корректировка аннотаций (меняем размер bboxes)
    new_bboxes = []
    for box in bboxes:
        class_id, x, y, bw, bh = box
        x, y, bw, bh = x * scale_factor, y * scale_factor, bw * scale_factor, bh * scale_factor
        new_bboxes.append([class_id, x, y, bw, bh])
    
    return resized_img, new_bboxes






import cv2
import numpy as np

def scale_image(image, bboxes, scale_factor):
    """Масштабирование изображения с сохранением YOLO-аннотаций (нормализованных)."""
    h, w = image.shape[:2]
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)

    # Масштабируем изображение
    resized_img = cv2.resize(image, (new_w, new_h))

    if scale_factor > 1:
        # Увеличение -> Обрезка центра
        crop_x = (new_w - w) // 2
        crop_y = (new_h - h) // 2
        final_img = resized_img[crop_y:crop_y + h, crop_x:crop_x + w]

        # Корректируем YOLO боксы (центр + размер)
        new_bboxes = []
        for class_id, x, y, bw, bh in bboxes:
            x = ((x * new_w) - crop_x) / w
            y = ((y * new_h) - crop_y) / h
            bw = (bw * new_w) / w
            bh = (bh * new_h) / h
            new_bboxes.append([class_id, x, y, bw, bh])

    else:
        # Уменьшение -> Паддинг (по центру)
        pad_x = (w - new_w) // 2
        pad_y = (h - new_h) // 2
        final_img = np.zeros((h, w, 3), dtype=np.uint8)
        final_img[pad_y:pad_y + new_h, pad_x:pad_x + new_w] = resized_img

        # Корректируем YOLO боксы (центр + размер)
        new_bboxes = []
        for class_id, x, y, bw, bh in bboxes:
            x = ((x * new_w) + pad_x) / w
            y = ((y * new_h) + pad_y) / h
            bw = (bw * new_w) / w
            bh = (bh * new_h) / h
            new_bboxes.append([class_id, x, y, bw, bh])

    return final_img, new_bboxes















# Загружаем все изображения
image_paths = glob(os.path.join(input_images_dir, "*.jpg"))

for image_path in image_paths:
    # Загружаем изображение
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Загружаем аннотации (YOLO format)
    label_path = os.path.join(input_labels_dir, os.path.basename(image_path).replace(".jpg", ".txt"))
    
    if not os.path.exists(label_path):
        continue  # Пропускаем, если нет аннотации
    
    with open(label_path, "r") as f:
        lines = f.readlines()
    
    bboxes = []
    for line in lines:
        values = line.split()
        class_id = int(values[0])
        x, y, w, h = map(float, values[1:])
        bboxes.append([class_id, x, y, w, h])
    
    #  Применяем аугментации
    augmented_images = []
    augmented_bboxes = []

    # 0 Горизонтальный флип
    flipped_img, flipped_boxes = flip_image(img, bboxes, flip_code=1)
    augmented_images.append(flipped_img)
    augmented_bboxes.append(flipped_boxes)

    # 1 Вертикальный флип
    flipped_img, flipped_boxes = flip_image(img, bboxes, flip_code=0)
    augmented_images.append(flipped_img)
    augmented_bboxes.append(flipped_boxes)

    # 2 Яркость
    bright_img = adjust_brightness(img, factor=1.4)
    augmented_images.append(bright_img)
    augmented_bboxes.append(bboxes)

    # 3 Яркость
    bright_img = adjust_brightness(img, factor=0.5)
    augmented_images.append(bright_img)
    augmented_bboxes.append(bboxes)

    # 4 Шум
    noisy_img = add_noise(img)
    augmented_images.append(noisy_img)
    augmented_bboxes.append(bboxes)


    # 5 Масштабирование
    scaled_img, scaled_boxes = scale_image(img, bboxes, scale_factor=1.4)
    augmented_images.append(scaled_img)
    augmented_bboxes.append(scaled_boxes)


    # 6  Масштабирование
    scaled_img, scaled_boxes = scale_image(img, bboxes, scale_factor=0.8)
    augmented_images.append(scaled_img)
    augmented_bboxes.append(scaled_boxes)



    # Сохраняем аугментированные изображения и аннотации
    for i, (aug_img, aug_boxes) in enumerate(zip(augmented_images, augmented_bboxes)):
        aug_boxes = clip_bbox(aug_boxes) 
        output_image_path = os.path.join(output_images_dir, f"{os.path.basename(image_path).replace('.jpg', '')}_aug{i}.jpg")
        cv2.imwrite(output_image_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))
        
        output_label_path = os.path.join(output_labels_dir, f"{os.path.basename(label_path).replace('.txt', '')}_aug{i}.txt")
        with open(output_label_path, "w") as f:
            for bbox in aug_boxes:
                f.write(f"{bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} {bbox[4]}\n")

print("Аугментированный датасет сохранён!")

Аугментированный датасет сохранён!
